## 14단계: 같은 변수 반복 사용

> 현재 DeZero는 같은 변수를 반복 사용할 경우 아래와 같은 문제가 발생합니다. \
$x = 3.0$으로 설정하고 $y = x + x$의 미분값을 구한다면 $y = 2x$이니 $\frac{\partial{y}}{\partial{x}} = 2$가 되어야 합니다.

<img src="images/그림 14-1.png" width=250/>

In [1]:
# 필요 모듈 정의

import numpy as np

class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.asarray(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def add(x0, x1):
    return Add()(x0, x1)

def square(x):
    return Square()(x)

In [ ]:
x = Variable(np.array(3.0))
y = add(x, x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)  # 미분값이 1로 계산됨

y 6.0
x.grad 1.0


### 14.1 문제의 원인

그 이유는 Variable 클래스에서 미분값을 덮어씌우기 때문이다. \
`그림 14-2`를 봤을 때, 올바른 미분값을 구하기 위해서는 **'덮어씌우기'**가 아닌 **'합'**을 구해야 한다.

<img src="images/그림 14-2.png" width=500/>

### 14.2 해결책

Variable 클래스에서 미분값을 전파하는 부분을 **'덮어씌우기'**에서 **'합'**으로 전환한다.

In [3]:
class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                # 기존 코드
                # x.grad = gx
                
                # 수정 코드
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    # 여기서 메모리 효율적인 복합 대입 연산자 '+='를 사용할 수도 있지만,
                    # 서로 다른 변수(Variable)들의 미분값(grad)이 같은 메모리주소의 값을 참조하게 될 수 있다.
                    # 해결하기 위해서는 ndarray.copy() 등의 메소드를 쓸 수도 있겠지만, 단순화를 위해 생략한다.
                    

                if x.creator is not None:
                    funcs.append(x.creator)

In [5]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)  # 미분값이 2.0으로 잘 나옴

2.0


In [6]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print(x.grad)  # 미분값이 3.0으로 잘 나옴

3.0


### 14.3 미분값 재설정

방금의 수정사항으로 새로운 주의사항이 나오는데, 바로 같은 변수로 여러 번 역전파를 수행할 때이다.

In [7]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)  # 예상 값: 2.0

# 두 번째 계산(같은 변수 x를 재사용)
y = add(add(x, x), x)
y.backward()
print(x.grad)  # 예상 값: 3.0

2.0
5.0


이 문제를 해결하기 위해 Variable에 미분값을 초기화 시켜주는 `cleargrad` 메서드를 추가해주자.

In [8]:
class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)
    
    # cleargrad 메서드 추가
    def cleargrad(self):
        # 단순히 미분값을 초기화함
        self.grad = None


# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)  # 예상 값: 2.0

# 두 번째 계산(같은 변수 x를 재사용)
x.cleargrad()  # 미분값 초기화
y = add(add(x, x), x)
y.backward()
print(x.grad)  # 예상 값: 3.0

2.0
3.0


이것으로 변수를 재사용할 때 발생하는 문제를 해결했다. \
그러나 아직 중요한 문제가 남아있는데, 이를 15, 16단계에서 해결하여 Variable 클래스를 완성하자.